# Accessing data from NASA Astrophysics Archives Using Python

NASA missions have collected a huge amount of data spanning a large range in wavelengths. These data are housed in four different archives: the [HEASARC](https://heasarc.gsfc.nasa.gov/), [MAST](https://archive.stsci.edu/), [IRSA](https://irsa.ipac.caltech.edu), [NED](https://ned.ipac.caltech.edu). However, different physical locations are not an obstacle for astronomers doing multiwavelength science projects that need to make use of data from different archives. The archives have been working together to make sure that you don’t need to know too much about *us* to assemble the data you need to get your research done. We’ve done this by standardizing the way that programs can access the data we house. Since Python is a very popular programming language, we are going to use it to show you how you can take advantage of this standardization to put together and analyze a multiwavelength data set. For this workshop, we are using a sample of interacting galaxies, and we will assemble a multiwavelength data set for it. You can generalize this example to much larger data sets of your own.

<a id='toc'></a>

<h2>Table of Contents</h2>

<a href='#availabledatasets'>1. Available Data Sets</a>

<a href='#sample'>2. Sample of Interacting Galaxies</a>

<a href = "#ned">3. Data Discovery -- NASA Extragalactic Database</a>

<a href = "#registry">4. Data Discovery -- The Registry</a><BR>

<a href = "#registryheasarc">4a. Search the registry for simple image access resources available from HEASARC</a><BR>

<a href = "#registryallwise">4b. Search the registry for AllWISE</a><BR>

<a href = "#registrymultiwavelength">4c. Define Base URLs for GALEX, AllWISE, and 2MASS.<BR>

<a href = "#simpleimageaccess">5. Simple Image Access</a>

<a href = "#ucd">6. Universal Common Descriptors (UCDs)</a>

<a href = "#fetch">7. Retrieve the images that you found</a>

<a href = "#multiwavelengthsearch">8. Search a list of data sets for images that cover a position on the sky</a><BR>

<a href = "#cutouts">9. Make Cutouts</a><BR>

<a href = "#simpleconesearch">10. Catalogs: Simple Cone Search</a><BR>

<a href = "#tableaccessprotocol">11. Catalogs: Table Access Protocol</a><BR>

<a href = "#simplespectralaccess">12. Simple Spectral Access</a><BR>


## To Do / Questions

### General
* With the new utility functions for VO queries, the NED queries are pretty complex in comparison.  
 * Should they still be the first things to be shown?  
 * Should they be in this main notebook?

* Find a way to shorten some of these output tables (as rendered in github).

### Image functionality
* consolidate coordiante handling among our query routines.
* VO queries expect ICRS coordinates.  If sc is not in ICRS, apply this transform:  sc = sc.transform_to('icrs')
* change input keyword from inradius to radius
* no need to use utils.sval to compute coords for params.
* in verbose mode, print all all params, ideally as they appear on the URL in the query.


<a id='availabledatasets'></a>

## 1. Available Data Sets

NASA Archives offer many publically available data sets over a large range in wavelengths. Below is a summary of what you can expect from each of the archives.

The **NASA/IPAC Extragalactic Database (NED)** is a comprehensive multiwavelength database for extragalactic objects, providing a systematic, ongoing fusion of information integrated from hundreds of large sky surveys and over a hundred thousand research publications. The contents and services span the entire observed spectrum from gamma rays through radio frequencies. As new observations are published, they are crossidentified with previous data and integrated into a unified database to simplify queries and retrieval.The LEVEL 5 Knowledgebase enhances review articles
in extragalactic astrophysics and cosmology with direct links from object names and graphical content to related database queries.

The **High Energy Astrophysics Science Archive Research Center (HEASARC)** is the primary archive for high-energy astronomy orbital missions observing at extreme ultraviolet, X-ray and gamma-ray wavelengths. Several of these missions, including Chandra, Fermi and XMM-Newton, fall under NASA's Physics of the Cosmos (PCOS) theme, while others, such as NICER, NuSTAR and Swift, are part of NASA's Explorers program. Since its merger with the Legacy Archive for Microwave Background Data Analysis (LAMBDA) in 2008, the HEASARC's holdings also include data from space-based missions, balloons, and ground-based facilities, such as COBE, WMAP, ACT, etc., that have studied the relic cosmic microwave background (CMB).

The **Mikulski Archive for Space Telescopes (MAST)** is a NASA-funded archive for a wide range of astronomical missions, primarily supporting space-based ultraviolet, optical and near-infrared telescopes. Best known as the archive for the Hubble Space Telescope (HST), MAST also archives data from other projects such as Kepler, GALEX, PanSTARRS, FUSE, and over 100 community-contributed collections of High-Level Science Products. Future missions will include the James Webb Space Telescope (JWST) and the Transiting Exoplanet Survey Satellite (TESS).

The **NASA/IPAC Infrared Sciece Archive (IRSA)** is chartered to curate the science products of NASA's infrared and submillimeter missions, as well as data from related projects. These include data from pointed observatories such as Spitzer and Herschel, as well as large-area and all-sky surveys such as 2MASS, WISE, IRAS, Planck, and AKARI. In total, IRSA provides access to more than 20 billion astronomical measurements, including all-sky coverage in 24 bands. 

<a href = "#toc">Table of Contents</a>

**TBD** Need label for this setup section

<a id='sample'></a>

In [50]:
# Astropy and requests imports
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
import requests

# Utility functions used in the workshop.
from workshop_utils.utils import print_registry_row, download_file

# Use the (in-progress) VO astroquery module
from navo_utils.registry import Registry
from navo_utils.image import Image, ImageColumn
from navo_utils.utils import astropy_table_from_votable_response


In [39]:
## Suppress unimportant warnings.
import warnings
warnings.filterwarnings("ignore")

## 2. Science Sample for Today: Interacting Galaxies

A HEASARC scientist is studying a sample of potentially interacting galaxies, and these will be the focus of our examples during this workshop. She has put the coordinates of these galaxies into a simple CSV file.

In [6]:
from astropy.io import ascii
list_of_galaxies = ascii.read('interacting_gals.csv', delimiter=',')  
print(list_of_galaxies)     

   ra     dec  
------- -------
9.90704 8.96507
19.0186 46.7304
20.2887 40.4703
125.886 21.3377
136.002 21.9679
141.057 40.6372
  146.7 22.0116
148.785 14.2922
149.751 17.8168
175.039  15.327
    ...     ...
241.519 20.8014
317.088 18.2002
329.235 6.64845
 333.83 37.3012
335.756 30.9249
344.396 19.7827
348.682 4.53407
 350.39 27.1181
350.438 9.07761
357.827 20.5778
0.36165 31.4334
Length = 30 rows


<a href = "#toc">Table of Contents</a>

<a id='ned'></a>

## 3. Data Discovery: NASA/IPAC Extragalactic Database (NED)

A common first step in extragalactic studies is to search NED to see what is already in the literature about your sample. In this example, we are going to do a quick SED search to find out which missions cover most of these galaxies. This will give us on overview of what kinds of data we might be interested in exploring in more detail.

Add a simple concrete example doing a NED example. Querying NED for SED. Talk about how other queries that NED might have, and consider adding a VO query.

Start with this simple cone search
http://vao.stsci.edu/directory/getRecord.aspx?id=ivo://ned.ipac/Basic_Data_Near_Position

**TBD** The access_url for that cone search is http://ned.ipac.caltech.edu/cgi-bin/NEDobjsearch?search_type=Near+Position+Search&of=xml_main&
Why is it mentioned here?

In [51]:
ned_base_url = 'http://vo.ned.ipac.caltech.edu'
ned_sed_query_path = '/services/querySED?REQUEST=queryData'

query_url = ned_base_url + ned_sed_query_path
search_radius = 5.0 / 3600.0

# Create a list of coordinate strings for input to the query.
# Each element needs to be a string that is parseable by SkyCoord.
skycoord_list = [str(row['ra'])+' '+str(row['dec']) for row in list_of_galaxies]

result_list = Image.query(coords=skycoord_list, radius=search_radius, service=access_url)

for i, result in enumerate(result_list):
    print(f'\nGalaxy {i}: {len(result)} results found for {skycoord_list[i]}')
    if len(result) > 0:
        # The column 'ACREF' contains the URL suffix for a NED SED query on the item.
        # Get the query suffix for the first item in the result.  Note that special 
        # characters need to be url-encoded.
        suffix = urllib.parse.quote(result[0]['ACREF'], safe = '/,&,?,=')
        ned_targetname_url = ned_base_url + suffix
        
        #Retrieve the SED using the Access URL, put it into an Astropy table,
        #And print the passbands in the SED.
        print(f'Querying {ned_targetname_url}')
        ned_data_response = requests.get(ned_targetname_url)
        if ned_data_response.status_code == 200:
            ned_data_table = astropy_table_from_votable_response(ned_data_response)
            print(ned_data_table['DataSpectralPassBand'])



    Querying service http://vo.ned.ipac.caltech.edu/services/querySED?REQUEST=queryData
    Got 1 results for parameters[0]
    Got 1 results for parameters[1]
    Got 2 results for parameters[2]
    Got 2 results for parameters[3]
    Got 6 results for parameters[4]
    Got 2 results for parameters[5]
    Got 1 results for parameters[6]
    Got 4 results for parameters[7]
    Got 6 results for parameters[8]
    Got 1 results for parameters[9]
    Got 1 results for parameters[10]
    Got 1 results for parameters[11]
    Got 3 results for parameters[12]
    Got 2 results for parameters[13]
    Got 2 results for parameters[14]
    Got 3 results for parameters[15]
    Got 2 results for parameters[16]
    Got 2 results for parameters[17]
    Got 3 results for parameters[18]
    Got 2 results for parameters[19]
    Got 3 results for parameters[20]
    Got 1 results for parameters[21]
    Got 2 results for parameters[22]
    Got 3 results for parameters[23]
    Got 1 results for parameters[2

DataSpectralPassBand
--------------------
      FUV (GALEX) AB
      FUV (GALEX) AB
      NUV (GALEX) AB
      NUV (GALEX) AB

Galaxy 15: 3 results found for 213.556 15.6214
Querying http://vo.ned.ipac.caltech.edu/services/accessSED?REQUEST=getData&TARGETNAME=SDSS%20J141413.34%2B153713.8
DataSpectralPassBand
--------------------
     u (SDSS PSF) AB
     g (SDSS PSF) AB
     r (SDSS PSF) AB
     i (SDSS PSF) AB
     z (SDSS PSF) AB

Galaxy 16: 2 results found for 219.967 42.7421
Querying http://vo.ned.ipac.caltech.edu/services/accessSED?REQUEST=getData&TARGETNAME=NGC%205730
DataSpectralPassBand
--------------------
     u (SDSS PSF) AB
   u (SDSS Model) AB
  u (SDSS CModel) AB
u (SDSS Petrosian)AB
             B (m_B)
           B (m_B^0)
                 m_p
 B (Johnson) Burrell
     g (SDSS PSF) AB
   g (SDSS Model) AB
                 ...
  J_14arcsec (2MASS)
  H_14arcsec (2MASS)
                   H
                   H
K_s_14arcsec (2MASS)
                 K_s
                 K_s

By perusing the above, we can see that there are several data sets that crop up again and again. For the sake of this workshop, we will concentrate on GALEX, WISE, and 2MASS.

## 4. Data Discovery -- The Registry

Archives register their data sets so that programs can discover them:
http://vao.stsci.edu/keyword-search/

The link above is a GUI, so you can play around with it. There is also an applications program interface (API) to this service, so that programs can send queries and retrieve search results automatically.  More details and examples can be found in *registry_prototype.ipynb* in this same directory.
### Imports for VO Queries 

<a id='registryheasarc'></a>

### 4a. Find all simple image access data sets from HEASARC.

In [ ]:
# Find all SIA services from HEASARC.
heasarc_image_services = Registry.query(source='heasarc', service_type='image')

heasarc_image_services.show_in_notebook()


### Exercise: Try replacing heasarc with irsa, stsci, ned in the above.

### Exercise: Try replacing image with cone, spectr, table in the above.

<a id='registryallwise'></a>

### 4b. Use the Registry to find the Base URL for AllWISE

In [ ]:
# New version
table = Registry.query(keyword='allwise', service_type='image')

print(f'{len(table)} result(s) found.')
for row in table:
    print_registry_row(row)

<a id='registrymultiwavelength'></a>

### 4c. Define Base URLs for GALEX, 2MASS, and AllWISE

In [ ]:
#Decide on missions you are interested in.
missions = ['GALEX', '2MASS', 'AllWISE']

base_urls = ['http://mast.stsci.edu/portal_vo/Mashup/VoQuery.asmx/SiaV1?MISSION=GALEX&amp;', 
             'http://irsa.ipac.caltech.edu/ibe/sia/twomass/allsky/allsky?',
             'https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?']

#Create an astropy table with this info.
search_list = Table([missions, base_urls], names = ('mission', 'access_url'))

#Show this table.
search_list.show_in_notebook()

<a id='registryreplace'></a>

<a href = "#toc">Table of Contents</a>

<a id='simpleimageaccess'></a>

## 5. Simple Image Access version 1

This method of searching for images is meant to be very simple. You specify a search position and size, and the service returns a list of candidate images in VO table format. Each candidate image has an access URL that you can use to grab that image in a second step.

More information on the protocol is here:
http://www.ivoa.net/documents/SIA/20091116/REC-SIA-1.0.html

In [ ]:
# Specify the query coordinates.  
# The query expects a SkyCoord, or any string that can be parsed in to a SkyCoord.
sc = SkyCoord(185.47873, 4.47365, unit=u.deg)

# Choose a search size. Zero means the image must contain the search point.
size = '0'  #diameter in degrees

# Perform the query on the 2MASS service.
result_list = Image.query(coords=sc, radius=size, service=search_list[2])

# The result is a list of tables, one for each coordinate in the query.
# In this case, we only input one coordinate, so there will only be one result table in the list.
print(f'Number of tables returned:  {len(result_list)}')
table = result_list[0]
table.show_in_notebook()


### Exercise: Modify the above to print the analogous GALEX table. You should find that the column names differ, and there are many additional file formats. 

<a href = "#toc">Table of Contents</a>

In [ ]:
# Get the important metadata for the result rows.

access_url_col = Image.get_column_name(table, ImageColumn.ACCESS_URL)
title_col = Image.get_column_name(table, ImageColumn.TITLE)
bandpass_col = Image.get_column_name(table, ImageColumn.BANDPASS)
for i, row in enumerate(table):
    print(f"""{i}:  {row[title_col]}
    Bandpass: {row[bandpass_col]} 
    {row[access_url_col]}
    """)

In [ ]:
table = Registry.query(source='stsci', keyword='hla', service_type='image')

print(f'{len(table)} result(s) found.')
for row in table:
    print_registry_row(row)

In [3]:
#Decide on missions you are interested in.
missions = ['GALEX', '2MASS', 'AllWISE', 'HLA']

base_urls = ['http://mast.stsci.edu/portal_vo/Mashup/VoQuery.asmx/SiaV1?MISSION=GALEX&amp;', 
             'http://irsa.ipac.caltech.edu/ibe/sia/twomass/allsky/allsky?',
             'https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?',
             'http://hla.stsci.edu/cgi-bin/hlaSIAP.cgi?imagetype=best&amp;inst=ACS,ACSGrism,WFC3,WFPC2,NICMOS,NICGRISM,COS,STIS,FOS,GHRS&amp;proprietary=false&amp;'
            ]

#Create an astropy table with this info.
search_list = Table([missions, base_urls], names = ('mission', 'access_url'))

#Show this table.
search_list.show_in_notebook()

idx,mission,access_url
0,GALEX,http://mast.stsci.edu/portal_vo/Mashup/VoQuery.asmx/SiaV1?MISSION=GALEX&amp;
1,2MASS,http://irsa.ipac.caltech.edu/ibe/sia/twomass/allsky/allsky?
2,AllWISE,https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?
3,HLA,"http://hla.stsci.edu/cgi-bin/hlaSIAP.cgi?imagetype=best&amp;inst=ACS,ACSGrism,WFC3,WFPC2,NICMOS,NICGRISM,COS,STIS,FOS,GHRS&amp;proprietary=false&amp;"


In [4]:
mission_results = []
for resource in search_list:
    # Specify the query coordinates.  
    # The query expects a SkyCoord, or any string that can be parsed in to a SkyCoord.
    sc = SkyCoord(185.47873, 4.47365, unit=u.deg)

    # Choose a search size. Zero means the image must contain the search point.
    size = '0'  #diameter in degrees

    # Perform the query on the 2MASS service.
    result_list = Image.query(coords=sc, radius=size, service=resource)
    mission_results.append(result_list)
    

    Querying service http://mast.stsci.edu/portal_vo/Mashup/VoQuery.asmx/SiaV1?MISSION=GALEX&


    Got 84 results for parameters[0]
    Querying service http://irsa.ipac.caltech.edu/ibe/sia/twomass/allsky/allsky?


    Got 3 results for parameters[0]
    Querying service https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?
    Got 4 results for parameters[0]
    Querying service http://hla.stsci.edu/cgi-bin/hlaSIAP.cgi?imagetype=best&inst=ACS,ACSGrism,WFC3,WFPC2,NICMOS,NICGRISM,COS,STIS,FOS,GHRS&proprietary=false&


    Got 20 results for parameters[0]


In [5]:
for i, mission in enumerate(mission_results):
    for j, search_coord_result in enumerate(mission):
        
        # Get interesting columns.
        access_url_col = Image.get_column_name(search_coord_result, ImageColumn.ACCESS_URL)
        title_col = Image.get_column_name(search_coord_result, ImageColumn.TITLE)
        format_col = Image.get_column_name(search_coord_result, ImageColumn.FORMAT)
        
        if len(search_coord_result) > 0:
            first_result = search_coord_result[0]
            access_url = first_result[access_url_col]
            out_path = download_file(access_url, directory='downloads', verbose=True)
            print(f'File written to: {out_path}')


download() verbose output:
    filename param: None
    content-disposition filename: None
    filename from URL: NGA_NGC4303-nd-int.fits.gz
    computed base_filename: NGA_NGC4303-nd-int.fits.gz
    computed full out_path: downloads/NGA_NGC4303-nd-int.fits.gz
        
File written to: downloads/NGA_NGC4303-nd-int.fits.gz

download() verbose output:
    filename param: None
    content-disposition filename: None
    filename from URL: ki0880080.fits.gz
    computed base_filename: ki0880080.fits.gz
    computed full out_path: downloads/ki0880080.fits.gz
        
File written to: downloads/ki0880080.fits.gz

download() verbose output:
    filename param: None
    content-disposition filename: None
    filename from URL: 1853p045_ac51-w3-int-3.fits
    computed base_filename: 1853p045_ac51-w3-int-3.fits
    computed full out_path: downloads/1853p045_ac51-w3-int-3.fits
        
File written to: downloads/1853p045_ac51-w3-int-3.fits

download() verbose output:
    filename param: None
    

In [ ]:
len(mission_results)

<a id='ucd'></a>

## 6. Find the Universal Common Descriptors for each column.

Each archive can name columns whatever they want, but the *type* of column has a universal name. To see the UCDs for the 2MASS allsky atlas images, try the following:

In [ ]:
for key in table.columns:
    col = table.columns[key]
    ucdval = col.meta.get('ucd')
    print(ucdval)
    print(key)
    print(' ')
 

The column with UCD = "VOX:Image_AccessReference" will have URLs that will lead us to images. The column with UCD = "VOX:BandPass_ID" will have the name of the filter. The column with UCD = "VOX:Image_Format" will tell us which products are FITS images.

<a href = "#toc">Table of Contents</a>

<a id='fetch'></a>

## 7. Retrieving images. 

Each candidate image has an Access URL that you can use to download the image. Here is a simple example for downloading an image. Update with http://docs.astropy.org/en/stable/api/astropy.utils.data.download_file.html

In [ ]:
import urllib.request
from astropy.utils.data import download_file

URL = "https://irsa.ipac.caltech.edu/ibe/data/twomass/allsky/allsky/000225s/s088/image/ki0880080.fits.gz"

urllib.request.urlretrieve(URL, "ki0880080.fits.gz")

<a href = "#toc">Table of Contents</a>

<a id='multiwavelengthsearch'></a>


## 8. For a given search position, loop through your list of missions and download the FITS files.

This can take a little bit of time. Aren't you glad it's scriptable?

In [ ]:
from astropy.table import Table, Column
import shutil
import ntpath

# Define the position and search size (degrees).
ra = '185.47873'
dec = '4.47365'

size = '0'  # Means the image must contain the search point

params = {'POS': ra + ',' + dec,
          'SIZE': size
         }

#Define some quantities you want to track.
outnames = []
missionnames = []
wavebands = []

print('Searching the following projects:')
print(search_list['mission'])
print(' ')

#For each Base URL in your list,
for i, base_url in enumerate(search_list['base_url']):

    #Send a URL request and get a table of images that meet your spatial constraints.
    print(' ')
    print('base_url:')
    print(base_url)
    r = requests.get(base_url, params)
    print(r.status_code)
    table=Table.read(io.BytesIO(r.content))

    #For each column in the returned table,
    for key in table.columns:
        
        #Get the data in the column.
        col = table.columns[key]
        
        #Get the universal content descriptor (UCD) for the column.
        ucdval = col.meta.get('ucd')
        
        #Find the columns of FITS images and get the images names
        if (ucdval is not None) and (ucdval == 'VOX:Image_Format'):
            imageformats = table[key]
        if (ucdval is not None) and (ucdval == 'VOX:Image_AccessReference'):
            imagenames = table[key]
        if (ucdval is not None) and (ucdval == 'VOX:BandPass_ID'):
            bandpasses = table[key]
             
    #For each imagename
    for j, imagename in enumerate(imagenames):

        #If the image is a FITS image,
        if ('image/fits' in imageformats[j].decode('utf-8')):

            #Track the corresponding mission name and waveband.
            missionname = search_list['mission'][i]
            waveband = bandpasses[j]
            
            #If it's an image we want (only AIS for GALEX),
            if (missionname != 'GALEX') or (missionname == 'GALEX' and 'AIS' in imagename.decode('utf-8')):
                
                #Send a request for this image.
                response = requests.get(imagename, stream=True)
                    
                #Remove the path from the URL to isolate the name we should output to file.
                outname = ntpath.basename(imagename)
                outnames.append(outname)
                print(outname)
                    
                #Write a FITS file.
                with open(outname, 'wb') as out_file:
                    shutil.copyfileobj(response.raw,out_file)

                #Keep track of which mission this image is associated with.
                missionnames.append(missionname)
                wavebands.append(waveband)

#Make an astropy table including all the tracked quantities.
downloaded_images = Table([missionnames, wavebands, outnames], names=('mission', 'wavebands', 'filename'))

#Show the table.
downloaded_images.show_in_notebook()

### Exercise: Keep track of the wavelength as well as the waveband, to enable wavelength sorting.

<a href = "#toc">Table of Contents</a>

<a id='cutouts'></a>

# 9. Make cutouts of the images you downloaded.

In [ ]:
import matplotlib.pyplot as plt
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
from astropy.wcs import wcs
import astropy.io.fits as fits
import numpy as np
import os
from astropy.io.fits import getdata, getheader
import astropy.units as u

#Set up the figure.
fig = plt.figure(figsize=(20,10))
plt.suptitle('POSITION = '+ra+', '+dec, fontsize=16)

#For each image,
i = 0
for filename in downloaded_images['filename']:
    
    #Make the title be the filename
    filename = downloaded_images['filename'][i]
    title = filename
    
    #Read in the image, header, and WCS
    data = getdata(filename, 0)
    header = getheader(filename, 0)
    w = wcs.WCS(header)

    #Make a cutout centered on the search position, with a 30 arcsec width.
    position = SkyCoord(ra, dec, unit = 'deg')
    size = 30.0 #* u.arcsec
    cutout = Cutout2D(data, position, (size, size), wcs=w)

    #Make a subplot for this image.
    ax = fig.add_subplot(3,4,i+1)
    ax.imshow(cutout.data, cmap='gray_r', origin='lower')
    ax.set_title(title)
    
    i = i + 1
    
plt.show()

### Exercise: Sort the images by wavelength. Remove GALEX single-epoch files. Add mission labels and wavebands.

<a href = "#toc">Table of Contents</a>

<a id='simpleconesearch'></a>

## 10. Catalogs: Simple Cone Search

The Simple Cone Search (SCS) protocol is used to search a catalog for entries with positions that fall within a given search region, specified by a position and cone search radius.

More information about SCS is available here:
http://www.ivoa.net/documents/latest/ConeSearch.html

Suppose we were considering proposing for observing time with Chandra. We might first want to check whether any of our sources were already observed by Chandra, XMM or ROSAT. Below, we show the search for the table 'chanmaster', the master catalog of observed Chandra targets, with a search radius of 0.1 degrees.

### Exercise: How would you figure out the table name 'chanmaster' for yourself? Hint: combine examples 4a and 4b above.

In [ ]:
#For each galaxy in our list,
for i in range(len(list_of_galaxies)):
    
    #Specify the Base URL of HEASARC's cone search service.
    Base_URL = 'https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl'
    
    #Specify the table to search, as well as the RA, DEC, and Search Radius that define the search region.
    params = {'table': 'chanmaster', 'RA': list_of_galaxies[i]['ra'], 'DEC':list_of_galaxies[i]['dec'], 'SR':0.1}

    #Send the request URL.
    r = requests.get(Base_URL, params=params)

    #Convert the results from VO Table to Astropy Table.
    table=Table.read(io.BytesIO(r.content))
    
    #Print the table.
    if (len(table) > 0):
        print('object: ', i)
        print(table)
        print('')

So we see that several galaxies in our sample have indeed already been observed, and we may wish to retrieve these images to see if they are deep enough to accomplish some of our science goals, or if we can justify additional exposures.

<a href = "#toc">Table of Contents</a>

<a id='tableaccessprotocol'></a>

## 11. Catalogs: Table Access Protocol (TAP)

We showed above how we can use NED to get the Spectral Energy Distribution of a galaxy. But if we wanted to investigate catalogs ourselves, we can use either the Simple Cone Search (SCS) Protocol above, or the Table Acess Protocol (TAP). SCS is easier to use, but limited to cone searches. TAP is more powerful, but at the cost of a bit of a learning curve. The registry examples above use TAP.

More information about TAP is available here:
http://www.ivoa.net/documents/TAP/

Let's use TAP to search the 2MASS catalog. Here we will use a synchronous query because we are doing a simple search that should return quickly. Users interested in searches that cover large areas or have complicated search criteria may wish to explore the asynchronous option.

In [ ]:
#Define the search URL. Select all columns from the 2MASS Point Source Catalog (fp_psc)
#for rows that lie within 0.01 deg of the specified search position. 
tap_base_url = "https://irsa.ipac.caltech.edu/TAP/sync?"

tap_params = {
    "request":"doQuery",  # Specify the request type
    "lang":"ADQL",        # Specify the language
#    "query": "SELECT TOP 1 * FROM fp_psc" #Give the query in that language
    "query":
        """SELECT ra,dec,j_m,h_m,k_m FROM fp_psc
        where CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',9.90704,8.96507,0.001))=1
        order by ra
        """
    }

# Send the search URL
r = requests.post(tap_base_url, data = tap_params)
#print(r.content)
# Convert the results into an astropy table.
table=Table.read(io.BytesIO(r.content))

#Print the table.
#table.pprint(max_width=-1)
#print(table['ra', 'dec', 'j_m', 'h_m', 'k_m'])
table.show_in_notebook()

<a href = "#toc">Table of Contents</a>

<a id='simplespectralaccess'></a>

## Simple Spectral Access (SSA)

The Simple Spectral Access (SSA) Protocol defines a uniform interface to remotely discover and access one dimensional spectra. 

More information about SSA is avialable here:
http://www.ivoa.net/documents/SSA/

Returning to our science example, suppose that we have analyzed the available Chandra image for the matches above and discovered that one of these galaxies appears to have a potential ultraluminous X-ray source (ULX)! Now, we want to charatcterize the spectrum for this source. Therefore we will ask for a large exposure time and need to justify this by showing the spectrum of a prototypical ULX, M82 X-1.

In [ ]:
import astropy.io.fits as fits
import astropy.coordinates as coord

m82=coord.SkyCoord.from_name("m82")
pos='{},{}'.format(m82.ra.deg,m82.dec.deg)

params = {'table': 'chanmaster',"POS":pos,"SIZE":".01", "REQUEST":"queryData"}

r = requests.get('https://heasarc.nasa.gov/xamin/vo/ssa', params=params)

spec_table = Table.read(io.BytesIO(r.content))
spec_table.show_in_notebook()

In [ ]:
## Get the first spectrum.
#The column SSA_reference is the one that contains the Access URL for the data.
hdu_list=fits.open(spec_table[0]['SSA_reference'].decode()) # Byte format, so just decode() to string

#Print the list of spectra.
table = Table(hdu_list[1].data)
table.show_in_notebook()


<a href = "#toc">Table of Contents</a>